# Agenda: `.loc` in Pandas

1. Retrieving from a series
    - Retrieving individual indexes
    - Don't forget `.iloc`, as well!
    - Boolean indexes / mask indexes
2. Retrieving from a data frame
    - Retrieving via the index
    - Retrieving via row + columns
3. More advanced topics
    - Using `lambda` with `.loc`

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
np.random.seed(0)  # make sure everyone has the same values
s = Series(np.random.randint(0, 1000, 10),
           index=list('abcdefghij'))
s

a    684
b    559
c    629
d    192
e    835
f    763
g    707
h    359
i      9
j    723
dtype: int64

In [4]:
# how can I retrieve from my series?
# the first thing you probably learned in working with Pandas is to use []

s['a']

np.int64(684)

# Don't use `[]` to retrieve from a series!

Why not?

1. It's a bad habit to get into, because when you get to data frames, it'll fail you
2. It's not nearly as powerful as `.loc`

# How do I retrieve?

1. `.iloc`
2. `.loc`

`.iloc` allows us to retrieve via the numeric position, just as we would do with a traditional Python data structure -- the indexes start at 0, and go up to len-1

In [6]:
s.iloc[0]

np.int64(684)

In [7]:
s.iloc[1]

np.int64(559)

In [8]:
s.iloc[-1]  # this returns the final element

np.int64(723)

In [9]:
s.iloc[3:7]  # from position 3 until (not including) position 7

d    192
e    835
f    763
g    707
dtype: int64

In [10]:
# most of the time, we're going to want to use .loc
# .loc works with the index as we've defined it